In [1]:
import ast
import pandas as pd
import numpy as np
from perceptronac.loading_and_saving import base_colors
from perceptronac.loading_and_saving import plot_comparison
from perceptronac.loading_and_saving import linestyle_tuple

In [2]:

def custom_df_to_dict(in_df):

    out_dict = dict()
    for hsh in set(str(in_df[c].values.tolist()) for c in in_df.columns):
        lbls = []
        for c in in_df.columns:
            if str(in_df[c].values.tolist()) == hsh:
                lbls.append(c)
        out_dict[",".join(lbls)] = ast.literal_eval(hsh.replace("nan","None"))
    
    return out_dict

In [3]:

def mohpo_grph(pth):

    df = pd.read_csv(pth,index_col=0)

    max_hv = df["max_hv"].iloc[0]

    df1 = (max_hv - df.drop("max_hv",axis=1)).applymap(np.log10)

    df2 = df1.rename(columns={"sobol_hv_list": "Sobol", 
    "ehvi_hv_list": "qNEHVI",
    "parego_hv_list": "qNParEGO",
    "c_angle_rule": "Alg.6",
    "u_angle_rule": "Alg.5",
    "c_gift_wrapping": "Alg.3",
    "u_gift_wrapping": "Alg.2",
    "c_tie_break": "Alg.4"})

    df3 = df2[sorted(df2.columns)]

    # df4 = df3.fillna(value=-1)
    # df4 = df3

    xvalues = df3.index
    data = custom_df_to_dict(df3)
    xlabel="number of observations"
    ylabel="Log Hypervolume Difference"
    xscale="linear"
    linestyles={k:ls for k,ls in zip(sorted(data.keys()),list(linestyle_tuple.values()))}
    colors={k:c for k,c in zip(sorted(data.keys()),list(base_colors.keys()))}
    markers = {k:"" for k in sorted(data.keys())}
    labels = {k:k for k in data.keys()}
    legend_ncol=1
    figsize=(6.4,4.8)
    new_order_sorted_keys=None
    legend_loc="lower left"

    fig = plot_comparison(
        xvalues,
        data,
        xlabel,
        ylabel,
        xscale,
        linestyles,
        colors,
        markers,
        labels,
        legend_ncol,
        figsize,
        new_order_sorted_keys,
        legend_loc
    )
    
    return fig

In [5]:

for x_axis in ["micro_joules_per_pixel","model_bits","params"]:
    
    fldr = "/home/lucas/Documents/perceptronac/scripts/tradeoffs/ax_results_energy_params_bits/"
    fil3 = f"{x_axis}_data_bits_over_data_samples_ax_methods_avgs.csv"
    pth = fldr + fil3
    
    fig = mohpo_grph(pth)
    
    fig.savefig(fil3.replace(".csv","_edited_graph.png"), dpi=300)